In [1]:
#!pip install surprise
#!pip install import-ipynb
import os
import csv
import pandas as pd
import numpy as np
import heapq
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import pickle
from surprise import accuracy
from collections import defaultdict
from surprise import KNNBasic
from collections import defaultdict
from operator import itemgetter
import NetflixLoadData as NetflixLoadData
import MovieCustomerInformation as information
#sns.set_style("darkgrid")

## Get data

In [2]:
use_pickle_file = True
max_n = 2500000 #how many rows we want from data_ratings and data_rating_plus_movie_title

in these txt files the movie_id's have been placed amongst the customer_id's, which means that we have to split that column into two, we do this by taking all ratings that are equal to NaN 
( meaning that we are now refering to a movie and not a customer ) and from there we split it into two columns.

In [3]:
data_movies, tmp_data_rating, tmp_data_rating_plus_movie_title, data_movies_categorized = NetflixLoadData.get_dataframes(use_pickle=use_pickle_file)

/mnt/24723F86723F5BAC/C Drif/ubuntu-github/netflix-recommendation/NetflixLoadData.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movies_categorize_cleaned["movie_year"] = data_movies_categorize_cleaned["movie_year"].astype(float)


In [4]:
data_rating, data_rating_plus_movie_title = NetflixLoadData.get_sample_of_data(tmp_data_rating, tmp_data_rating_plus_movie_title, max_n=max_n)

## Helper functions

In [5]:
def save_to_pickle(name, df):
    path_name = "pickle/"+name+".pickle"
    pickle_file = open(path_name,"wb")
    pickle.dump(df, pickle_file)
    pickle_file.close()

def load_pickle(name):
    path_name = "pickle/"+name+".pickle"
    return_input = open(path_name, "rb")
    return pickle.load(return_input)

## Working with the data

### Details about all customers/movies

In [6]:
# get the average movie rating for all customers
# used to determine if this user typically gives bad or good reviews
# and then we can see if he really hates or loves a movie
all_customers_average_ratings =  information.all_average_ratings(df=data_rating, type='customer_id')
print(all_customers_average_ratings)

       customer_id rating       avg_rating
                      sum count           
0                6   45.0    13   3.461538
1                7   83.0    18   4.611111
2               10   16.0     5   3.200000
3               25    3.0     1   3.000000
4               33    4.0     1   4.000000
...            ...    ...   ...        ...
360194     2649401   11.0     3   3.666667
360195     2649404    7.0     2   3.500000
360196     2649409   15.0     4   3.750000
360197     2649426   47.0    11   4.272727
360198     2649429   23.0     6   3.833333

[360199 rows x 4 columns]


In [7]:
all_movies_average_rating = information.all_average_ratings(df=data_rating, type='movie_id')
print(all_movies_average_rating)

    movie_id  rating       avg_rating
                 sum count           
0          1  2051.0   547   3.749543
1          2   516.0   145   3.558621
2          3  7326.0  2012   3.641153
3          4   389.0   142   2.739437
4          5  4468.0  1140   3.919298
..       ...     ...   ...        ...
463      464   226.0    82   2.756098
464      465   292.0   100   2.920000
465      466  1357.0   362   3.748619
466      467  3759.0  1055   3.563033
467      468  4062.0  1158   3.507772

[468 rows x 4 columns]


In [8]:
information.get_avg_rating_less_than(df=all_customers_average_ratings , max_rating=4)

       customer_id rating       avg_rating
                      sum count           
0                6   45.0    13   3.461538
2               10   16.0     5   3.200000
3               25    3.0     1   3.000000
5               42   14.0     4   3.500000
6               59   25.0     7   3.571429
...            ...    ...   ...        ...
360193     2649388   22.0     7   3.142857
360194     2649401   11.0     3   3.666667
360195     2649404    7.0     2   3.500000
360196     2649409   15.0     4   3.750000
360198     2649429   23.0     6   3.833333

[207379 rows x 4 columns]


In [9]:
information.get_avg_rating_higher_than(df=all_customers_average_ratings, min_rating=4)

       customer_id rating       avg_rating
                      sum count           
1                7   83.0    18   4.611111
13             126    5.0     1   5.000000
15             134   83.0    18   4.611111
19             168    9.0     2   4.500000
27             199   76.0    18   4.222222
...            ...    ...   ...        ...
360181     2649311   29.0     7   4.142857
360184     2649328   21.0     5   4.200000
360186     2649335   41.0    10   4.100000
360190     2649375   30.0     7   4.285714
360197     2649426   47.0    11   4.272727

[92187 rows x 4 columns]


In [10]:
information.get_avg_rating_less_than(df=all_movies_average_rating, max_rating=4)

    movie_id  rating       avg_rating
                 sum count           
0          1  2051.0   547   3.749543
1          2   516.0   145   3.558621
2          3  7326.0  2012   3.641153
3          4   389.0   142   2.739437
4          5  4468.0  1140   3.919298
..       ...     ...   ...        ...
463      464   226.0    82   2.756098
464      465   292.0   100   2.920000
465      466  1357.0   362   3.748619
466      467  3759.0  1055   3.563033
467      468  4062.0  1158   3.507772

[443 rows x 4 columns]


In [11]:
information.get_avg_rating_higher_than(df=all_movies_average_rating, min_rating=4)

    movie_id    rating        avg_rating
                   sum  count           
12        13     569.0    125   4.552000
31        32    7549.0   1854   4.071737
32        33   28722.0   6890   4.168650
67        68    9180.0   2216   4.142599
75        76   12083.0   2954   4.090386
84        85    1131.0    266   4.251880
105      106    3852.0    940   4.097872
134      135    1589.0    384   4.138021
166      167   37448.0   9151   4.092230
174      175  362148.0  90450   4.003847
208      209   18195.0   4373   4.160759
214      215   23632.0   5817   4.062575
222      223  101595.0  24125   4.211192
240      241  179297.0  43122   4.157901
252      253   20543.0   5050   4.067921
269      270  156266.0  36431   4.289369
315      316   10841.0   2520   4.301984
324      325    4703.0   1121   4.195361
344      345   17165.0   4175   4.111377
358      359   19027.0   4580   4.154367
420      421    3991.0    987   4.043566
429      430    6258.0   1546   4.047865
450      451    

In [12]:
count_movies_rated_by_customer = information.all_get_rated_count(df=data_rating, type="customer_id")
count_movies_rated_by_customer

,customer_id,movie_id
0,6,13
1,7,18
2,10,5
3,25,1
4,33,1
...,...,...
360194,2649401,3
360195,2649404,2
360196,2649409,4
360197,2649426,11


In [13]:
# def generate_random_user_ids(from_n, to_n):
#     ids = np.unique(data_rating['customer_id'])[from_n:to_n]
#     print("random user id used while in development\n")
#     return ids
# generate_random_user_ids(1000, 1500)

### Details about a specific customer

In [47]:
# ids I like -> 28812, 56514, 56520, 1488844
customer_id_use = 6 #84100

In [48]:
#all_custumers_id_rows(customer_id=customer_id_use)
## same as line above but including movie_year and movie_title
all_customers_id_plus_movie_title_rows(customer_id=customer_id_use)

,movie_id,customer_id,rating,movie_year,movie_title
187297,30,6,3.0,2003.0,Something's Gotta Give
539827,157,6,3.0,2004.0,Laird: White Knuckle Extreme
576723,173,6,4.0,1968.0,The Devil's Brigade
649632,175,6,5.0,1992.0,Reservoir Dogs
795442,191,6,2.0,2003.0,X2: X-Men United
881625,197,6,3.0,2004.0,Taking Lives
1053784,241,6,3.0,1959.0,North by Northwest
1368432,295,6,4.0,1995.0,Ace Ventura: When Nature Calls
1438379,299,6,3.0,2001.0,Bridget Jones's Diary
1723378,329,6,4.0,1999.0,Dogma


In [49]:
get_users_avg_rating(customer_id=customer_id_use)

0    3.461538
Name: avg_rating, dtype: float64

In [50]:
display_movies_customer_rated_higher_than(customer_id=customer_id_use, min_rating=4)

                             movie_title  rating
movie_id                                        
173                  The Devil's Brigade     4.0
175                       Reservoir Dogs     5.0
295       Ace Ventura: When Nature Calls     4.0
329                                Dogma     4.0
457                    Kill Bill: Vol. 2     5.0
average rating 0    3.461538
Name: avg_rating, dtype: float64


In [51]:
display_movies_customer_rated_lower_than(customer_id=customer_id_use, max_rating=4)

                                        movie_title  rating
movie_id                                                   
30                           Something's Gotta Give     3.0
157                    Laird: White Knuckle Extreme     3.0
191                                X2: X-Men United     2.0
197                                    Taking Lives     3.0
241                              North by Northwest     3.0
299                           Bridget Jones's Diary     3.0
361       The Phantom of the Opera: Special Edition     3.0
445                                 Michael Collins     3.0
average rating 0    3.461538
Name: avg_rating, dtype: float64


In [52]:
get_users_loved_hated_movies(customer_id=customer_id_use, minmax_rating=4)

User 6 loved these movies
The Devil's Brigade
Reservoir Dogs
Ace Ventura: When Nature Calls
Dogma
Kill Bill: Vol. 2

and disliked these movies
Something's Gotta Give
Laird: White Knuckle Extreme
X2: X-Men United
Taking Lives
North by Northwest
Bridget Jones's Diary
The Phantom of the Opera: Special Edition
Michael Collins


In [53]:

data_rating

,movie_id,customer_id,rating
0,1,1488844,3.0
1,1,822109,5.0
2,1,885013,4.0
3,1,30878,4.0
4,1,823519,3.0
...,...,...,...
2499995,468,1404021,5.0
2499996,468,544108,3.0
2499997,468,1071584,4.0
2499998,468,256747,3.0


## Recommendations with collaborative filtering

In [23]:
def build_training_set(df):
    reader = Reader()
    svd = SVD()
    data_new = Dataset.load_from_df(df[['customer_id', 'movie_id', 'rating']], reader)
    trainset = data_new.build_full_trainset()
    svd.fit(trainset)
    return trainset, svd

def get_summary(type="movie_id"):
    #>IF MOVIE:  movie_id, movie rating count, movie rating mean
    #>IF CUSTOMER: customer_id, custumer rating count, customer rating mean
    df_count_mean_summary = data_rating.groupby(type)['rating'].agg(['count', 'mean'])
    df_count_mean_summary.index = df_count_mean_summary.index.map(int)
    #>IF MOVIE: returns 1799.0 as a benchmark number
    #IF CUSTOMER: returns 52.0 as a benchmark number
    benchmark = round(df_count_mean_summary['count'].quantile(0.7),0)
    # drop all rows below benchmark
    df_drop_list = df_count_mean_summary[df_count_mean_summary['count'] < benchmark]
    # return all indexes to drop
    return df_drop_list

def get_customer_recommendations(customer_id, predictor):
    #> returns movie_id, movie_year, movie_title
    chosen_customer_pred = data_movies.copy()
    # fails if movie_id is the index so we have to reset the index back to normal (0-N)
    chosen_customer_pred = chosen_customer_pred.reset_index()
    # makes sure that we only pick movies that are not in the movie dropped list
    chosen_customer_pred = chosen_customer_pred[~chosen_customer_pred['movie_id'].isin(df_movie_drop_list)]
    # make prediction for customer with id = <customer_id> and put it into 'estimated_score'
    chosen_customer_pred['estimated_score'] = chosen_customer_pred['movie_id'].apply(lambda x: predictor.predict(customer_id, x).est)
    # sort by 'estimated score'
    chosen_customer_pred = chosen_customer_pred.sort_values('estimated_score', ascending=False).set_index('movie_id')
    return chosen_customer_pred

def display_rated_content(customer_id=customer_id_use, number_to_show=20):
    print("Movies/TV Shows rated by customer", customer_id)
    df = all_customers_id_plus_movie_title_rows(customer_id=customer_id)
    df = df[['movie_title', 'rating']].sort_values('rating', ascending=False)
    print(df[0:number_to_show].set_index('movie_title'))

def display_customers_recommendations(customer_id=customer_id_use, df=[], number_to_show=20):
    print("Movies/TV Shows recommended to customer")
    print(df[['movie_title', 'estimated_score']][0:number_to_show].set_index('movie_title'))

def display_history_plus_recommended(customer_id=customer_id_use, number_to_show=25):
    display_rated_content(customer_id=customer_id, number_to_show=number_to_show)
    chosen_customer_pred = get_customer_recommendations(customer_id=customer_id, predictor=svd)
    display_customers_recommendations(customer_id=customer_id, df=chosen_customer_pred, number_to_show=number_to_show)

def get_evaluation_and_svd(use_pickle=True):
    evaluationData_file_name = "evaluationData_build_training_set"
    svd_file_name = "svd_build_training_set"
    if(use_pickle == True):
        evaluationData = load_pickle(evaluationData_file_name)
        svd = load_pickle(svd_file_name)
        return evaluationData, svd
    else:
        evaluationData, svd = build_training_set(data_rating)
        save_to_pickle(evaluationData_file_name, evaluationData)
        save_to_pickle(svd_file_name, svd)
        return evaluationData, svd

In [24]:
evaluationData, svd = get_evaluation_and_svd(use_pickle=use_pickle_file)

In [25]:
## creating a list of the movies that we don't wanna include
df_movie_drop_list = get_summary(type="movie_id")
#df_customer_drop_list = get_summary(type="customer_id")

In [26]:
display_history_plus_recommended(customer_id=79724, number_to_show=10)

Movies/TV Shows rated by customer 79724
                            rating
movie_title                       
Dogma                          5.0
What the #$*! Do We Know!?     4.0
Reservoir Dogs                 4.0
Ed Wood                        3.0
Pay It Forward                 3.0
Igby Goes Down                 3.0
Elephant                       2.0
Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
Aqua Teen Hunger Force: Vol. 1                             5.000000
Lord of the Rings: The Return of the King: Exte...         4.490710
Rudolph the Red-Nosed Reindeer                             4.346710
ABC Primetime: Mel Gibson's The Passion of the ...         4.163294
The Weather Underground                                    4.129848
Boycott                                                    4.119695
Zatoichi's Conspiracy                                      4.

In [27]:
display_history_plus_recommended(customer_id=customer_id_use, number_to_show=10)

Movies/TV Shows rated by customer 84100
                        rating
movie_title                   
Something's Gotta Give     5.0
House of Sand and Fog      5.0
X2: X-Men United           4.0
Bridget Jones's Diary      4.0
Kill Bill: Vol. 2          4.0
Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
Lord of the Rings: The Return of the King: Exte...         4.657058
Spitfire Grill                                             4.183466
Zatoichi's Conspiracy                                      4.096977
Aqua Teen Hunger Force: Vol. 1                             4.050542
ABC Primetime: Mel Gibson's The Passion of the ...         4.047677
Inspector Morse 31: Death Is Now My Neighbour              4.040900
Dinosaur Planet                                            3.982314
Lilo and Stitch                                            3.953918
The Rise and Fall of EC

In [28]:
#TODO:
#for the new user problem
## find highest rated movie / tv shows that have been watched the most -> how likely is new user to click on it ?

## SOMETHING

In [29]:
from surprise import dataset
from surprise import KNNBaseline
from surprise.model_selection import train_test_split 
from surprise.model_selection import LeaveOneOut
from surprise import accuracy

In [30]:
def build_training_set_same_as_above(df):
    reader = Reader()
    svd = SVD()
    data_new = Dataset.load_from_df(df[['customer_id', 'movie_id', 'rating']], reader)
    trainset = data_new.build_full_trainset()
    svd.fit(trainset)
    return trainset, svd

def print_evaluation_accuracy(prediction):
    print("\nEvaluating accuracy of model...")
    print("RMSE: ", accuracy.rmse(prediction, verbose=False))
    print("MAE: ", accuracy.mae(prediction, verbose=False))

In [31]:
reader = Reader(line_format='user item rating', rating_scale=(1, 5))
class convert_to_raw_ratings(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in zip(df['customer_id'], df['movie_id'], df['rating'])]
        self.reader=reader

raw_ratings = convert_to_raw_ratings(data_rating, reader)
print("\nBuilding recommendation model...")
trainSet, testSet = train_test_split(raw_ratings, test_size=.25, random_state=1)


Building recommendation model...


In [32]:
algo = SVD(random_state=10)
algo.fit(trainSet)

print("\nComputing recommendations...")
predictions = algo.test(testSet)


Computing recommendations...


In [33]:
#cross_validate(algo,  raw_ratings, cv=5, verbose=True)

In [34]:
print_evaluation_accuracy(predictions)


Evaluating accuracy of model...
RMSE:  0.9912543460061926
MAE:  0.7797843476506224


In [35]:
print_evaluation_accuracy(svd.test(testSet))


Evaluating accuracy of model...
RMSE:  1.0386634945860738
MAE:  0.8522688604750867


## other

In [36]:
reader = Reader(line_format='user item rating', rating_scale=(1, 5))
dataset = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
trainset = dataset.build_full_trainset()

In [37]:
tmp_data_movies = data_movies[['movie_id', 'movie_title']]
tmp_data_movies = tmp_data_movies.set_index('movie_id').T
tmp_data_movies = tmp_data_movies.to_dict('list')
tmp_data_movies = {k: str(v[0]) for k,v in tmp_data_movies.items()}

In [38]:
similarity_matrix = KNNBasic(sim_options={'name': 'cosine', 'user_based': False}).fit(trainset).compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [39]:
# generate_random_user_ids(1000, 1500)

In [40]:
# for i in generate_random_user_ids(1000, 1500):
#     if (len(all_custumers_id_rows(i)) > 3):
#         print(i)

In [41]:
test_subject = customer_id_use#16272
k = 10

In [42]:
test_subject_iid = trainset.to_inner_uid(test_subject)

test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbours = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [43]:
candidates = defaultdict(float)

for itemID, rating in k_neighbours:
    try:
        similarities = similarity_matrix[itemID]
        for innerID, score in enumerate(similarities):
            candidates[innerID] += score * (rating / 5.0)
    except:
        continue

In [44]:
def getMovieName(movieID):
  if int(movieID) in tmp_data_movies:
    return tmp_data_movies[int(movieID)]
  else:
      return ""

In [45]:
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
    watched[itemID] = 1

recommendations = []
position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        recommendations.append(getMovieName(trainset.to_raw_iid(itemID)))
        position += 1
        # only want top n which in our case in 10
        if(position > 10): break

for rec in recommendations:
    print("Movie: ", rec)


Movie:  Lord of the Rings: The Return of the King: Extended Edition: Bonus Material
Movie:  Mississippi Burning
Movie:  North by Northwest
Movie:  Silkwood
Movie:  Rabbit-Proof Fence
Movie:  Woman of the Year
Movie:  Elfen Lied
Movie:  The Chorus
Movie:  Sex and the City: Season 4
Movie:  Parenthood
Movie:  Charlotte's Web


In [46]:
# #TODO: keep or throw away since it's never used
# sim_options = {'name': 'pearson_baseline', 'user_based': False}
# simsAlgo = KNNBaseline(sim_options=sim_options)
# simsAlgo.fit(evaluationData)